In [1]:
!pip install gensim

In [2]:
#!pip install tensorly

#imported packages
import os
import numpy as np
import pandas as pd
import tensorly as tl
from tensorly.decomposition import tucker
import math
from scipy import stats
import csv
from matplotlib.pyplot import figure
import matplotlib
from sklearn.metrics import mean_squared_error as rmse
import time

# LDA for clusterring
import gensim
from gensim import corpora
import matplotlib.pyplot as plt

In [4]:
#defined functions
#data column methods

#convert data to array
def getseqid(targetval):
    #convert taz idx
    targetidx = {}
    idxtarget = {}
    idx = 0
    for i in targetval:
        targetidx[i] = idx        
        idxtarget[idx] = i
        idx += 1
    return(targetidx, idxtarget)

def data2arr(datapath, binaryidx=True, to2d=True):
    taz2idx = lambda x: tazidx[x]
    str2time = lambda x: int(int(x.split(':')[0])*12 + int(x.split(':')[1])/5)
    timelen = 24*12 #with 5-min interval
    newid2idx = lambda x: newididx[x]
    
    datapath = raw_dt_path[0]
    dt = pd.read_csv(datapath)
    
    #convert time and location
    tazidx, idxtaz = getseqid(np.unique(dt['taz']))
    dt['tazidx'] = dt['taz'].apply(taz2idx)
    dt['timeidx'] = dt['time'].apply(str2time)

    newididx, idxnewid = getseqid(np.unique(dt['newid']))
    dt['newididx'] = dt['newid'].apply(newid2idx)
    
    if to2d == True:
        dt['ltidx'] = dt['tazidx']*timelen+dt['timeidx']
        nrow = len(newididx)
        ncol = timelen * len(tazidx)

        arr = np.zeros((nrow, ncol))
        dt = dt[['newididx','ltidx', 'sum']].to_numpy()

        #assign binary index
        for i, j, count in dt:
            if binaryidx == True:
                arr[i, j] = 1
            else:
                arr[i, j] = count
        return(arr)
    else:
        nrow = len(newididx)
        ntaz = len(tazidx)
        ntime = timelen
        
        arr = np.zeros((nrow, ntaz, ntime))
        dt = dt[['newididx', 'tazidx', 'timeidx', 'sum']].to_numpy()
        
        for i, j, k, count in dt:
            if binaryidx == True:
                arr[i, j, k] = 1
            else:
                arr[i, j, k] = sum
        return(arr)
    

In [5]:
#process the fcd data
path = "G:/My Drive/2021/Bias/sumo_simulation/"
#os.listdir()

raw_dt_path = []
for i in os.listdir(path):
    if len(i) == 12 and '2017' in i:
        raw_dt_path.append(path+i)

In [15]:
#!pip install fastdtw
#clustering
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

In [ ]:
# Generate synthetic binary time series data
# Replace this with your actual data
# Calculate pairwise DTW distances
def get_dtw_dist(arr, return_mx):
    
    num_samples = arr.shape[0]
    dtw_distances = np.zeros((num_samples, num_samples))
    #print(dtw_distances.shape)
    for i in range(num_samples):
        for j in range(i, num_samples):
            distance, _ = fastdtw(arr[i], arr[j], dist=euclidean)
            dtw_distances[i, j] = distance
            dtw_distances[j, i] = distance

    # Normalize DTW distances (optional but can help with K-Means)
    max_distance = np.max(dtw_distances)
    normalized_dtw_distances = dtw_distances / max_distance
    if return_mx == True:
        return(dtw_distances, normalized_dtw_distances)
    else:
        return(normalized_dtw_distances)

def conduct_cluster(normalized_dtw_distances, num_clusters):
    # Apply K-Means clustering
    #num_clusters = 3  # Adjust as needed
    #normalized_dtw_distances = get_dtw_dist(arr, return_mx)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(normalized_dtw_distances)
    # Print cluster labels
    #print("Cluster Labels:")
    #print(cluster_labels)
    db_index = davies_bouldin_score(normalized_dtw_distances, cluster_labels)
    return(db_index)

In [ ]:
#check the daily pattern across all the taz
cluster_num = np.arange(2, 11)
arr_test = data2arr(raw_dt_path[0])

db_score = []
dtw_mx, normalized_dtw = get_dtw_dist(arr_test, True)
for num_c in cluster_num:
    db_score.append(normalized_dtw, num_c)

(13709, 13709)


In [ ]:
#check the daily pattern for each individual 
arr_test = data2arr(raw_dt_path[0], to2d=False)


db_scroe_test = []
for i in arr_test.shape[0]:
    dtw_mx_, normalized_dtw_ = get_dtw_dist(arr_test[i], True)
    for num_c in cluster_num:
        db_score_test.append(normalized_dtw_, num_c)
    plt.plot(db_score_test)
    plt.title(f'individual idx = {i}')
    


In [21]:

db_index = davies_bouldin_score(normalized_dtw_distances, cluster_labels)

print("Davies-Bouldin Index:", db_index)

Davies-Bouldin Index: 2.53114765553973


In [7]:
arr.shape

(13709, 167, 288)

In [8]:
arr_reshaped = arr.reshape(arr.shape[0], -1)

In [12]:
#arr_reshaped = arr.reshape(arr.shape[0], -1)

# Convert the binary data into a document-term matrix
documents = [list(map(str, row)) for row in arr_reshaped]

# Create a dictionary and a corpus for Gensim
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Apply LDA-like topic modeling
lda_model = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
topics = lda_model.print_topics(num_words=10)

# Print the discovered topics
for topic in topics:
    print(topic)

(0, '1.000*"0.0" + 0.000*"1.0"')
(1, '0.999*"0.0" + 0.001*"1.0"')
(2, '1.000*"0.0" + 0.000*"1.0"')
(3, '1.000*"0.0" + 0.000*"1.0"')
(4, '1.000*"0.0" + 0.000*"1.0"')


starttime = time.time()

rank_individuals = 10
rank_locations = 10
rank_times = 12

core, factors = tucker(arr, rank=[rank_individuals, rank_locations, rank_times])
endtime = time.time()
print(f'running time {endtime-starttime}')

reconstructed_data = tl.tucker_to_tensor((core, factors))